시작하기전에 Subsampling부터 한다. `Qiime2`에서는 feature_table을 만들때는 가능하지만 데이터를 처음 불러올때는 해당 기능이 없는것 같다.

# seqtk으로 subsampling

일단은 10,000개 정도면 적당할 것으로 보인다. [seqtk](https://github.com/lh3/seqtk)는 깃헙에서 사용법을 확인 할 수 있다.

- Subsample 10000 read pairs from two large paired FASTQ files (remember to use the same random seed to keep pairing):

```
seqtk sample -s100 read1.fq 10000 > sub1.fq
seqtk sample -s100 read2.fq 10000 > sub2.fq
```  

## conda 명령어로 설치

```
conda install -c bioconda seqtk
```

In [1]:
!conda install -c bioconda seqtk -y

Solving environment: done

## Package Plan ##

  environment location: /home/partrita/miniconda3/envs/qiime2-2019.7

  added / updated specs:
    - seqtk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         152 KB
    openssl-1.1.1d             |       h7b6447c_1         3.7 MB
    seqtk-1.3                  |       hed695b0_2          39 KB  bioconda
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW packages will be INSTALLED:

  seqtk              bioconda/linux-64::seqtk-1.3-hed695b0_2

The following packages will be UPDATED:

  openssl            conda-forge::openssl-1.1.1c-h516909a_0 --> pkgs/main::openssl-1.1.1d-h7b6447c_1

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                              

In [8]:
!ls -a data

.		02-Man_2.fastq	05-Man_1.fastq	07-Man_2.fastq	10-Man_1.fastq
..		03-Man_1.fastq	05-Man_2.fastq	08-Man_1.fastq	10-Man_2.fastq
01-Man_1.fastq	03-Man_2.fastq	06-Man_1.fastq	08-Man_2.fastq
01-Man_2.fastq	04-Man_1.fastq	06-Man_2.fastq	09-Man_1.fastq
02-Man_1.fastq	04-Man_2.fastq	07-Man_1.fastq	09-Man_2.fastq


In [3]:
!seqtk sample -s100 data/01-Man_1.fastq 10000 > sub_data/01-Man_1.fastq
!seqtk sample -s100 data/01-Man_2.fastq 10000 > sub_data/01-Man_2.fastq
!seqtk sample -s100 data/02-Man_1.fastq 10000 > sub_data/02-Man_1.fastq
!seqtk sample -s100 data/02-Man_2.fastq 10000 > sub_data/02-Man_2.fastq
!seqtk sample -s100 data/03-Man_1.fastq 10000 > sub_data/03-Man_1.fastq
!seqtk sample -s100 data/03-Man_2.fastq 10000 > sub_data/03-Man_2.fastq
!seqtk sample -s100 data/04-Man_1.fastq 10000 > sub_data/04-Man_1.fastq
!seqtk sample -s100 data/04-Man_2.fastq 10000 > sub_data/04-Man_2.fastq
!seqtk sample -s100 data/05-Man_1.fastq 10000 > sub_data/05-Man_1.fastq
!seqtk sample -s100 data/05-Man_2.fastq 10000 > sub_data/05-Man_2.fastq
!seqtk sample -s100 data/06-Man_1.fastq 10000 > sub_data/06-Man_1.fastq
!seqtk sample -s100 data/06-Man_2.fastq 10000 > sub_data/06-Man_2.fastq
!seqtk sample -s100 data/07-Man_1.fastq 10000 > sub_data/07-Man_1.fastq
!seqtk sample -s100 data/07-Man_2.fastq 10000 > sub_data/07-Man_2.fastq
!seqtk sample -s100 data/08-Man_1.fastq 10000 > sub_data/08-Man_1.fastq
!seqtk sample -s100 data/08-Man_2.fastq 10000 > sub_data/08-Man_2.fastq
!seqtk sample -s100 data/09-Man_1.fastq 10000 > sub_data/09-Man_1.fastq
!seqtk sample -s100 data/09-Man_2.fastq 10000 > sub_data/09-Man_2.fastq
!seqtk sample -s100 data/10-Man_1.fastq 10000 > sub_data/10-Man_1.fastq
!seqtk sample -s100 data/10-Man_2.fastq 10000 > sub_data/10-Man_2.fastq

위의 명령어에서 `-s100`는 seed number로 동일한 샘플링을 위해서 동일한 숫자를 사용해야한다.

# qza 파일 만들기

`qiime2`를 사용하기 위해서는 데이터를 `.qza`형식으로 변환해야한다. 아래 명령어를 사용하기 위해 `manifest`파일을 만들고 TSV형식으로 Sample ID와 file_path를 지정해줘야 한다.



In [4]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path sub_manifest \
  --output-path sub_10k_demux.qza \
  --input-format PairedEndFastqManifestPhred33V2

Imported sub_manifest as PairedEndFastqManifestPhred33V2 to paired_end_demux.qza


demultiplexing을 결과를 다음 명령어로 확인한다. 

- to determine how many sequences were obtained per sample
- to get a summary of the distribution of sequence qualities at each position in your sequence data.


In [6]:
!qiime demux summarize \
  --i-data sub_10K_demux.qza \
  --o-visualization output/sub_demux.qzv

Saved Visualization to: output/sub_demux.qzv


In [83]:
from qiime2 import Visualization
Visualization.load('output/sub_demux.qzv')

<visualization: Visualization uuid: d5a19487-77b9-4b0f-8681-36b9886cf674>

# Sequence quality control 

위의 **quality plot** 결과를 볼때 forward는 22-290bp, reverse는 6-217bp 정도가 쓸만한 것 같다.

그런데 foward와 reverse의 최소값이 35bp인데 뭔가 문제가 있는 것 같다.

## TODO: 서열의 분표를 확인하고 outlier는 제거하자

In [25]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs sub_10k_demux.qza \
  --p-n-threads 0 \
  --p-trim-left-f 35 \
  --p-trim-left-r 35 \
  --p-trunc-len-f 260 \
  --p-trunc-len-r 260 \
  --o-table output/sub_table.qza \
  --o-representative-sequences output/sub_rep_seqs.qza \
  --o-denoising-stats output/sub_denoising_stats.qza

Saved FeatureTable[Frequency] to: output/sub_table.qza
Saved FeatureData[Sequence] to: output/sub_rep_seqs.qza
Saved SampleData[DADA2Stats] to: output/sub_denoising_stats.qza


# feature table summary


In [54]:
!qiime feature-table summarize \
  --i-table output/sub_table.qza \
  --o-visualization output/sub_table.qzv \
  --m-sample-metadata-file metadata.tsv

Saved Visualization to: output/sub_table.qzv


In [82]:
Visualization.load('output/sub_table.qzv')

<visualization: Visualization uuid: df1bbe5c-1774-41b9-a57f-a830c1b266d7>

In [56]:
!qiime feature-table tabulate-seqs \
  --i-data output/sub_rep_seqs.qza \
  --o-visualization output/sub_rep_seqs.qzv

Saved Visualization to: output/sub_rep_seqs.qzv


In [81]:
Visualization.load('output/sub_rep_seqs.qzv')

<visualization: Visualization uuid: ee6fb663-7780-46f1-9a63-fbf2745e631f>

denoising stat도 시각화 한다.

In [58]:
!qiime metadata tabulate \
  --m-input-file output/sub_denoising_stats.qza \
  --o-visualization output/sub_denoising_stats.qzv

Saved Visualization to: output/sub_denoising_stats.qzv


In [80]:
Visualization.load('output/sub_denoising_stats.qzv')

<visualization: Visualization uuid: 2b328279-37a9-401b-9307-6f018e01ec67>

# Generate a tree for phylogenetic diversity analyses

In [60]:
!qiime fragment-insertion sepp \
  --i-representative-sequences output/sub_rep_seqs.qza \
  --o-tree output/sub_tree.qza \
  --o-placements output/sub_tree_placements.qza \
  --p-threads 8  # update to a higher number if you can

Saved Phylogeny[Rooted] to: output/sub_tree.qza
Saved Placements to: output/sub_tree_placements.qza


# Alpha and beta diversity analysis

- we must normalize our data to account for uneven sequencing depth between samples.

We’ll use qiime diversity alpha-rarefaction to subsample the ASV table at different depths (between --p-min-depth and --p-max-depth) and calculate the alpha diversity using one or more metrics (--p-metrics). When we checked the feature table, we found that the sample with the fewest sequences in the denoised table has `85` features and that the sample with the most has `4996` features. We want to set a maximum depth close to the maximum number of sequences. We also know that if we look at a sequencing depth around `4250` sequences per sample, we’ll be looking at information from 22 samples. So, let’s set this as our maximum sequencing depth.


> max-depth가 2800일때 Retained 120 (45.28%) features in 4 (40.00%) samples


This visualizer computes one or more alpha diversity metrics at multiple sampling depths, in steps between 1 (optionally controlled with --p-min-depth) and the value provided as --p-max-depth. At each sampling depth step, 10 rarefied tables will be generated, and the diversity metrics will be computed for all samples in the tables. 

In [61]:
!qiime diversity alpha-rarefaction \
  --i-table output/sub_table.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/sub_alpha_rarefaction_curves.qzv \
  --p-max-depth 2800

Saved Visualization to: output/sub_alpha_rarefaction_curves.qzv


In [79]:
Visualization.load('output/sub_alpha_rarefaction_curves.qzv')

<visualization: Visualization uuid: 57111cc6-a1fd-451f-b518-f23930ac4f88>

In [63]:
!qiime diversity core-metrics-phylogenetic \
  --i-table output/sub_table.qza \
  --i-phylogeny output/sub_tree.qza \
  --m-metadata-file metadata.tsv \
  --p-sampling-depth 1500 \
  --output-dir output/core-metrics-results

Usage: qiime diversity core-metrics-phylogenetic [OPTIONS]

  Applies a collection of diversity metrics (both phylogenetic and non-
  phylogenetic) to a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table containing the samples over which
                          diversity metrics should be computed.     [required]
  --i-phylogeny ARTIFACT  Phylogenetic tree containing tip identifiers that
    Phylogeny[Rooted]     correspond to the feature identifiers in the table.
                          This tree can contain tip ids that are not present
                          in the table, but all feature ids in the table must
                          be present in this tree.                  [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)        The total frequency that each sample should be
                          rarefied to prior to computing diversity metrics.
                                           

# alpha diversity

In [64]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity output/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/core-metrics-results/faiths_pd_statistics.qzv

Saved Visualization to: output/core-metrics-results/faiths_pd_statistics.qzv


In [72]:
Visualization.load('output/core-metrics-results/faiths_pd_statistics.qzv')

<visualization: Visualization uuid: 1d797a29-1a43-45d2-b258-f3012ef903df>

메타데이터에 categorical 그룹을 추가했다.

# Taxonomic classification

For this analysis, we’ll use a pre-trained naive Bayes machine-learning classifier that was trained to differentiate taxa present in the 99% Greengenes 13_8 reference set trimmed to 250 bp of the V4 hypervariable region

In [65]:
!wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza"

--2019-09-18 09:41:14--  https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza [following]
--2019-09-18 09:41:15--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.212.248
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.212.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28373760 (27M) [application/x-www-form-urlencoded]
Saving to: `gg-13-8-99-515-806-nb-classifier.qza'

100%[======================================>] 28,373,760  5.66M/s   in 5.3s    

2019-09-18 09:41:21 (5.10 MB/s) - `gg-13-

In [66]:
!qiime feature-classifier classify-sklearn \
  --i-reads output/sub_rep_seqs.qza  \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --o-classification output/sub_taxonomy.qza

Saved FeatureData[Taxonomy] to: output/sub_taxonomy.qza


In [67]:
!qiime metadata tabulate \
  --m-input-file output/sub_taxonomy.qza \
  --o-visualization output/sub_taxonomy.qzv

Saved Visualization to: output/sub_taxonomy.qzv


In [68]:
!qiime feature-table tabulate-seqs \
  --i-data output/sub_rep_seqs.qza \
  --o-visualization output/sub_rep_set.qzv

Saved Visualization to: output/sub_rep_set.qzv


# bar chart

In [69]:
!qiime feature-table filter-samples \
  --i-table output/sub_table.qza \
  --p-min-frequency 1500 \
  --o-filtered-table output/table_1k.qza

Saved FeatureTable[Frequency] to: output/table_1k.qza


In [70]:
!qiime taxa barplot \
  --i-table output/table_1k.qza \
  --i-taxonomy output/sub_taxonomy.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/taxa_barplot.qzv

Saved Visualization to: output/taxa_barplot.qzv


In [78]:
Visualization.load('output/taxa_barplot.qzv')

<visualization: Visualization uuid: b531399e-eb10-4dd5-860f-27f138442b3e>

# 엽록체 제외한 시각화

https://docs.qiime2.org/2019.7/tutorials/filtering/ 에서 데이터 필터링하는 방법이 있음.

In [74]:
!qiime taxa filter-table \
  --i-table output/table_1k.qza\
  --i-taxonomy output/sub_taxonomy.qza  \
  --p-exclude chloroplast \
  --o-filtered-table output/no_chloroplast.qza

Saved FeatureTable[Frequency] to: output/no_chloroplast.qza


In [75]:
!qiime taxa barplot \
  --i-table output/no_chloroplast.qza \
  --i-taxonomy output/sub_taxonomy.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/no_taxa_barplot.qzv

Saved Visualization to: output/no_taxa_barplot.qzv


In [85]:
Visualization.load('output/no_taxa_barplot.qzv')

<visualization: Visualization uuid: 3e95ac14-725a-4f65-b9c8-fa5361091c40>